In [8]:
%pip install wandb -q
import wandb
wandb.login()

     |████████████████████████████████| 1.4MB 5.3MB/s 
     |████████████████████████████████| 163kB 39.3MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 112kB 35.8MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
import torch
from torchtext import data
SEED = 0

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [10]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 197k/84.1M [00:00<00:47, 1.77MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 75.3MB/s]


In [11]:
print(f"Data points in training dataset : {len(train_data)}")
print(f"Data points in testing dataset : {len(test_data)}")

Data points in training dataset : 25000
Data points in testing dataset : 25000


In [12]:
print(vars(train_data.examples[0]))

{'text': ['As', 'a', 'horse', 'lover', 'one', 'can', 'only', 'appreciate', 'this', 'movie', '.', 'There', 'are', 'few', 'movies', 'that', 'show', 'horsemanship', 'as', 'this', 'one', 'does', '.', 'I', 'would', 'love', 'to', 'know', 'if', 'Brian', 'does', 'his', 'own', 'riding', 'in', 'the', 'film', '.', 'Would', 'also', 'like', 'to', 'know', 'if', 'he', 'enjoys', 'horses', '.', 'Brian', 'has', 'been', 'in', 'a', 'lot', 'of', 'movies', 'where', 'he', 'has', 'ridden', '.', 'Where', 'did', 'he', 'learn', 'to', 'ride', '?', 'The', 'only', 'part', 'that', 'is', 'hard', 'for', 'me', 'to', 'take', 'is', 'that', 'the', 'riding', 'scenes', 'are', 'always', 'full', 'tilt', ',', 'like', 'a', 'horse', 'can', 'run', 'forever', 'at', 'full', 'steam', '.', 'The', 'camera', '-', 'work', 'is', 'first', 'rate', 'and', 'captures', 'the', 'horses', 'in', 'a', 'way', 'that', 'shows', 'how', 'dangerous', 'things', 'can', 'be', 'on', 'top', 'of', 'a', 'horse', '.', 'It', 'would', 'be', 'very', 'interesting',

In [13]:
vocab_size = 25000

TEXT.build_vocab(train_data, max_size = vocab_size)
LABEL.build_vocab(train_data)

In [14]:
print(f"Unique tokens in TEXT vocabulary : {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary : {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary : 25002
Unique tokens in LABEL vocabulary : 2


In [15]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 289838), (',', 275296), ('.', 236843), ('and', 156483), ('a', 156282), ('of', 144055), ('to', 133886), ('is', 109095), ('in', 87676), ('I', 77546), ('it', 76545), ('that', 70355), ('"', 63329), ("'s", 61928), ('this', 60483), ('-', 52863), ('/><br', 50935), ('was', 50013), ('as', 43508), ('with', 42807)]


In [16]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [17]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7faf2bc41510>, {'neg': 0, 'pos': 1})


In [19]:
vocab_size = 25000

TEXT.build_vocab(train_data, max_size = vocab_size)
LABEL.build_vocab(train_data)

In [20]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [21]:
BATCH_SIZE = 64

device = torch.device('cuda' 
                      if torch.cuda.is_available() 
                      else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
(train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [22]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.RNN(embedding_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, text):
    # text = [sentence_length, batch_size]

    embedded = self.embedding(text)

    # embedded = [sentence_length, batch_size, embedding_dim]

    output, hidden = self.rnn(embedded)

    # output = [sentence_length, batch_size, hidden_dim]
    # hidden = [1, batch_size, hidden_dim]

    assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
    return self.fc(hidden.squeeze(0))

In [23]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [27]:
import torch.optim as optim
#wandb.config.lr = 0.01
optimizer = optim.SGD(model.parameters(), lr = 1e-03)

In [28]:
criterion = nn.BCEWithLogitsLoss()

In [29]:
model = model.to(device)
criterion = criterion.to(device)

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [30]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [32]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
import torch.optim as optim
import time

In [35]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [36]:
# Initialize WandB 
wandb.init(project='text_classification_project', 
           entity='pratikraut_')

wandb.watch(model)

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train Loss": train_loss,
        "Train Acc": train_acc,
        "Valid Loss": valid_loss,
        "Valid Acc": valid_acc})

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.66%
	 Val. Loss: 0.694 |  Val. Acc: 49.76%
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.693 | Train Acc: 49.59%
	 Val. Loss: 0.694 |  Val. Acc: 49.72%
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.693 | Train Acc: 49.42%
	 Val. Loss: 0.694 |  Val. Acc: 50.24%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.72%
	 Val. Loss: 0.694 |  Val. Acc: 50.03%
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 0.693 | Train Acc: 50.01%
	 Val. Loss: 0.694 |  Val. Acc: 50.20%
Time Elapsed : 14.1031s


In [39]:

import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [40]:
predict_sentiment(model, "This film is terrible")

0.43235504627227783

In [41]:
predict_sentiment(model, 'this film is good')

0.46270737051963806